## Advent of Code 2022
### Day 5
#### Part 1

In [6]:
import pandas as pd
from itertools import islice

In [3]:
day = pd.read_csv('data/day05.txt',sep= " ", names=['dummy',"num_move","dummy2","move_from","dummy3","move_to"],skiprows=10)

day.head()

,dummy,num_move,dummy2,move_from,dummy3,move_to
0,move,3,from,1,to,2
1,move,1,from,7,to,1
2,move,1,from,6,to,5
3,move,5,from,5,to,9
4,move,2,from,5,to,2


In [53]:
instructions_df = day.drop(columns=["dummy","dummy2","dummy3"])
instructions_df.head()

,num_move,move_from,move_to
0,3,1,2
1,1,7,1
2,1,6,5
3,5,5,9
4,2,5,2


In [7]:
with open('data/day05.txt') as f:
    orig_stacks = list(islice(f, 9))
orig_stacks

['[N] [G]                     [Q]    \n',
 '[H] [B]         [B] [R]     [H]    \n',
 '[S] [N]     [Q] [M] [T]     [Z]    \n',
 '[J] [T]     [R] [V] [H]     [R] [S]\n',
 '[F] [Q]     [W] [T] [V] [J] [V] [M]\n',
 '[W] [P] [V] [S] [F] [B] [Q] [J] [H]\n',
 '[T] [R] [Q] [B] [D] [D] [B] [N] [N]\n',
 '[D] [H] [L] [N] [N] [M] [D] [D] [B]\n',
 ' 1   2   3   4   5   6   7   8   9 \n']

In [49]:
def create_empty_stack_dicts(stacks_list):
    """ 
    Generate dictionary of number of stacks to store
        stack contents in
    """

    stack_num_idx = len(stacks_list) - 1
    ids_str = stacks_list[stack_num_idx]
    ids_str = ids_str.replace(" ","")
    ids_str = ids_str.replace("\n","")
    #print(ids_str)

    stack_dicts = {int(i):[] for i in ids_str}
    #print(stack_dicts)

    return stack_dicts

def gen_stack_row(str_stack, num_stacks):
    """ 
    Generates a list of stack contents for a single row
        from a string representation of that row
    """

    str_stack = str_stack.replace("\n","")
    inp_len = len(str_stack)
    chunk_size = round(inp_len / num_stacks)
    #print(chunk_size)
    stack_sep =[str_stack[i:i+chunk_size].strip().replace("[","").replace("]","") \
        for i in range(0, inp_len, chunk_size)]
    #print(stack_sep)

    return stack_sep

def create_stacks(stacks_list):
    """ 
    Create dictionary of list representation of stacks 
        from list of string representations.
    """
    # generate empty dictionary of lists for stacks
    stack_dicts = create_empty_stack_dicts(stacks_list)
    num_stacks = len(stack_dicts)

    max_stack_height = len(stacks_list) - 1
    #print('max height',max_stack_height)
    for r in range(max_stack_height-1,-1,-1):
        stack_row = stacks_list[r]
        #print('stack_row:',stack_row)
        stack_list = gen_stack_row(stack_row, num_stacks)
        for idx, val in enumerate(stack_list):
            #print(idx, val)
            if val != "":
                stack_dicts[idx+1].append(val)

    return stack_dicts


In [68]:
stacks_dict = create_stacks(orig_stacks)
stacks_dict

{1: ['D', 'T', 'W', 'F', 'J', 'S', 'H', 'N'],
 2: ['H', 'R', 'P', 'Q', 'T', 'N', 'B', 'G'],
 3: ['L', 'Q', 'V'],
 4: ['N', 'B', 'S', 'W', 'R', 'Q'],
 5: ['N', 'D', 'F', 'T', 'V', 'M', 'B'],
 6: ['M', 'D', 'B', 'V', 'H', 'T', 'R'],
 7: ['D', 'B', 'Q', 'J'],
 8: ['D', 'N', 'J', 'V', 'R', 'Z', 'H', 'Q'],
 9: ['B', 'N', 'H', 'M', 'S']}

In [69]:
def one_move(num_move, move_from, move_to, stacks_dict):
    """ 
    Follow one step of the rearrangement procedure
    """

    for _ in range(num_move):
        tomove = stacks_dict[move_from].pop()
        stacks_dict[move_to].append(tomove)
    
    return stacks_dict

def move_boxes(instructions_df, stacks_dict, move_func):
    """ 
    Perform the entire rearrangement procedure
    """

    for _, num_move, move_from, move_to in instructions_df.itertuples():
        stacks_dict = move_func(num_move, move_from, move_to, stacks_dict)

    return stacks_dict

In [70]:
final_stacks = move_boxes(instructions_df, stacks_dict, one_move)
final_stacks

{1: ['G'],
 2: ['R'],
 3: ['V', 'S', 'T', 'T'],
 4: ['D', 'N', 'R', 'W', 'S'],
 5: ['F', 'V', 'B', 'T', 'B', 'J', 'D', 'M', 'W'],
 6: ['Q', 'D', 'B', 'Q', 'P', 'Q', 'F', 'M', 'V', 'N'],
 7: ['N', 'J'],
 8: ['J', 'B', 'N', 'T', 'L', 'H', 'M', 'D', 'R', 'D', 'H', 'N', 'H'],
 9: ['N', 'V', 'Q', 'R', 'S', 'B', 'B', 'Z', 'H', 'Q', 'H']}

In [71]:
# id top crate from each stack
# without removing them from the stack
top_crates = ""
for i in range(len(final_stacks)):
    stack_height = len(final_stacks[i+1])
    top_crate = final_stacks[i+1][stack_height-1]
    top_crates = top_crates + top_crate

print(top_crates)

GRTSWNJHH


#### Part 2

In [72]:
def one_move_9001(num_move, move_from, move_to, stacks_dict):
    """ 
    Follow one step of the rearrangement procedure
        for the 9001 crane (moving multiple boxes at once)
    """

    # id boxes to move
    tomove = stacks_dict[move_from][-num_move:]
    # remove boxes from original stack
    stacks_dict[move_from] = stacks_dict[move_from][:-num_move]
    # add boxes to new stack
    stacks_dict[move_to].extend(tomove)

    return stacks_dict

In [73]:
stacks_dict = create_stacks(orig_stacks)
stacks_dict

{1: ['D', 'T', 'W', 'F', 'J', 'S', 'H', 'N'],
 2: ['H', 'R', 'P', 'Q', 'T', 'N', 'B', 'G'],
 3: ['L', 'Q', 'V'],
 4: ['N', 'B', 'S', 'W', 'R', 'Q'],
 5: ['N', 'D', 'F', 'T', 'V', 'M', 'B'],
 6: ['M', 'D', 'B', 'V', 'H', 'T', 'R'],
 7: ['D', 'B', 'Q', 'J'],
 8: ['D', 'N', 'J', 'V', 'R', 'Z', 'H', 'Q'],
 9: ['B', 'N', 'H', 'M', 'S']}

In [74]:
final_stacks_9001 = move_boxes(instructions_df, stacks_dict, one_move_9001)
final_stacks_9001

{1: ['Q'],
 2: ['L'],
 3: ['M', 'H', 'R', 'F'],
 4: ['S', 'T', 'M', 'J', 'Q'],
 5: ['D', 'S', 'Q', 'N', 'V', 'T', 'H', 'W', 'D'],
 6: ['D', 'J', 'N', 'R', 'H', 'B', 'B', 'F', 'T', 'B'],
 7: ['N', 'B'],
 8: ['J', 'P', 'Q', 'Q', 'N', 'S', 'W', 'R', 'V', 'V', 'T', 'D', 'H'],
 9: ['R', 'V', 'G', 'D', 'B', 'Z', 'H', 'N', 'N', 'B', 'M']}

In [75]:
# id top crate from each stack
# without removing them from the stack
top_crates_9001 = ""
for i in range(len(final_stacks_9001)):
    stack_height = len(final_stacks_9001[i+1])
    top_crate = final_stacks_9001[i+1][stack_height-1]
    top_crates_9001 = top_crates_9001 + top_crate

print(top_crates_9001)

QLFQDBBHM


#### References Used

https://stackoverflow.com/questions/40648345/read-first-n-lines-using-readlines